<a href="https://colab.research.google.com/github/ashagedo/MSCI-641-project/blob/main/2a_DistilBART_Inference_with_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import os
import torch
import nltk
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, TrainerCallback
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
import pandas as pd
from tqdm import tqdm
nltk.download('punkt')
nltk.download('wordnet')

In [2]:
# Import the files.upload() function from the google.colab module
from google.colab import files

# Upload files using the files.upload() function
uploaded_files = files.upload()

# Access the uploaded files
for filename in uploaded_files.keys():
    print('Uploaded file:', filename)
    with open(filename, 'r') as file:
        train_file = file.read()

Saving train.jsonl to train.jsonl
Uploaded file: train.jsonl


In [3]:
# Import the files.upload() function from the google.colab module
from google.colab import files

# Upload files using the files.upload() function
uploaded_files = files.upload()

# Access the uploaded files
for filename in uploaded_files.keys():
    print('Uploaded file:', filename)
    with open(filename, 'r') as file:
        val_file = file.read()

Saving val.jsonl to val.jsonl
Uploaded file: val.jsonl


In [4]:
# Import the files.upload() function from the google.colab module
from google.colab import files

# Upload files using the files.upload() function
uploaded_files = files.upload()

# Access the uploaded files
for filename in uploaded_files.keys():
    print('Uploaded file:', filename)
    with open(filename, 'r') as file:
        test_file = file.read()

Saving test.jsonl to test.jsonl
Uploaded file: test.jsonl


In [5]:
#Custom Model with Classification Head, Structure Optimized for Classification

class CustomModel(nn.Module):
    def __init__(self, model_name, num_labels, class_weights=None, classification_weight=None, summarization_weight=None):
        super(CustomModel, self).__init__()
        self.transformer = BartForConditionalGeneration.from_pretrained(model_name)
        self.classification_head = nn.Linear(self.transformer.config.d_model, num_labels)

        self.class_weights = class_weights
        self.classification_weight = classification_weight
        self.summarization_weight = summarization_weight

        # Optional: Add dropout for regularization
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_ids, attention_mask=None, labels=None, decoder_input_ids=None, classification_labels=None):
        # Get outputs from the transformer model
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_input_ids=decoder_input_ids)
        encoder_output = self.dropout(outputs.encoder_last_hidden_state[:, 0, :])
        logits = self.classification_head(encoder_output)

        # Compute classification loss if labels are provided
        classification_loss = None
        if classification_labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
            classification_loss = loss_fct(logits, classification_labels)

        # The loss returned by the transformer model (summarization task loss)
        summarization_loss = outputs.loss

        # Combine the losses with specified weights
        if classification_loss is not None:
            loss = (self.classification_weight * classification_loss) + (self.summarization_weight * summarization_loss)
        else:
            loss = summarization_loss

        return {
            "loss": loss,
            "classification_loss": classification_loss,
            "summarization_loss": summarization_loss,
            "logits": logits,
            "encoder_last_hidden_state": outputs.encoder_last_hidden_state,
            "past_key_values": outputs.past_key_values,
            "decoder_hidden_states": outputs.decoder_hidden_states,
            "decoder_attentions": outputs.decoder_attentions,
            "cross_attentions": outputs.cross_attentions,
            "encoder_hidden_states": outputs.encoder_hidden_states,
            "encoder_attentions": outputs.encoder_attentions,
        }

    def save_pretrained(self, save_directory):
        self.transformer.save_pretrained(save_directory)
        classification_head_path = os.path.join(save_directory, 'classification_head.bin')
        torch.save(self.classification_head.state_dict(), classification_head_path)

    @classmethod
    def from_pretrained(cls, model_name_or_path, num_labels, class_weights=None):
        model = cls(model_name_or_path, num_labels, class_weights)
        model.transformer = BartForConditionalGeneration.from_pretrained(model_name_or_path)
        classification_head_path = os.path.join(model_name_or_path, 'classification_head.bin')
        model.classification_head.load_state_dict(torch.load(classification_head_path))
        return model

#Pre-process Data
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)# Remove special characters, but keep some punctuation
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
# Load data into DataFrames
train_data = pd.read_json(train_file, lines=True)
val_data = pd.read_json(val_file, lines=True)
test_data = pd.read_json(test_file, lines=True)


In [ ]:
# Prepare test data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_path='/content/drive/MyDrive/DistilBART_weighted_for_classificationx1_10epochs_20percent/epoch_10.0'
num_labels = 3
class_weights=None

tokenizer = BartTokenizer.from_pretrained(model_path)
custom_model = CustomModel(model_path, num_labels, class_weights=class_weights, classification_weight=1.0, summarization_weight=1.0)
custom_model.to(device)

# Preprocess and tokenize test data
#test_data['concatenated_text'] = test_data.apply(lambda row: ' '.join(row['postText']) + ' ' + ' '.join(row['targetTitle']) + ' ' + ' '.join(row['targetParagraphs']), axis=1)
postText_label = "postText"
targetTitle_label = "targetTitle"
targetParagraphs_label = "targetParagraphs"
test_data['concatenated_text'] = test_data.apply(
    lambda row: f"{postText_label} {' '.join(row['postText'])} "
                f"{targetTitle_label} {' '.join(row['targetTitle'])} "
                f"{targetParagraphs_label} {' '.join(row['targetParagraphs'])}",
    axis=1
)

test_inputs = test_data['concatenated_text'].tolist()
test_inputs = [preprocess_text(text) for text in test_inputs]
test_inputs = tokenizer(test_inputs, return_tensors='pt', padding=True, truncation=True, max_length=1024)  # Adjusted max_length

# Move the tokenized inputs to the GPU
test_input_ids = test_inputs['input_ids'].to(device)
test_attention_mask = test_inputs['attention_mask'].to(device)

# Create a DataLoader for the test data
batch_size = 2  # Adjust batch size based on your GPU memory
test_dataset = torch.utils.data.TensorDataset(test_input_ids, test_attention_mask)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

# Lists to store predictions
classification_predictions = []
summary_predictions = []

custom_model.eval()
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = batch

        # Ensure inputs are on the correct device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Text Summarization
        try:
            summary_ids = custom_model.transformer.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=50,
                num_beams=4,
                early_stopping=True
            )
            summary_texts = [
                tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True) for ids in summary_ids
            ]
            summary_predictions.extend(summary_texts)
        except Exception as e:
            print(f"Error generating summary: {e}")
            summary_texts = [""] * input_ids.size(0)
            summary_ids = torch.zeros((input_ids.size(0), 1), dtype=torch.long).to(device)
            summary_predictions.extend(summary_texts)

        # Classification
        outputs = custom_model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=summary_ids)
        logits = outputs['logits']
        predicted_label_idxs = torch.argmax(logits, dim=-1).tolist()

        label_map = {0: 'phrase', 1: 'passage', 2: 'multi'}
        classification_predictions.extend([label_map[idx] for idx in predicted_label_idxs])

# Save predictions to DataFrame
test_data['spoiler'] = summary_predictions
test_data['spoilerType'] = classification_predictions
output_file = 'DistilBART_weighted_for_classificationx1_10epochs_20percent.csv'
test_data[['id', 'spoilerType', 'spoiler']].to_csv(output_file, index=False)
test_data[['id', 'spoilerType', 'spoiler']].to_csv('DistilBART_weighted_for_classificationx1_10epochs_20percent.csv', index=False)
print(f"Predictions saved to {output_file}")


In [8]:
#Prepare Training and Validation Data for Model Evaluation
max_len = 1024
batch_size = 2
num_labels = 3
label_map = {'phrase': 0, 'passage': 1, 'multi': 2}

#train_data['concatenated_text'] = train_data.apply(lambda row: ' '.join(row['postText']) + ' ' + ' '.join(row['targetTitle']) + ' ' + ' '.join(row['targetParagraphs']), axis=1)
postText_label = "postText"
targetTitle_label = "targetTitle"
targetParagraphs_label = "targetParagraphs"
train_data['concatenated_text'] = train_data.apply(
    lambda row: f"{postText_label} {' '.join(row['postText'])} "
                f"{targetTitle_label} {' '.join(row['targetTitle'])} "
                f"{targetParagraphs_label} {' '.join(row['targetParagraphs'])}",
    axis=1
)
train_inputs = train_data['concatenated_text'].tolist()
train_inputs = [preprocess_text(text) for text in train_inputs]
train_inputs = tokenizer(train_inputs, return_tensors='pt', padding=True, truncation=True, max_length=1024)  # Adjusted max_length

train_summaries = train_data['spoiler'].tolist()
train_labels = torch.tensor(train_data['tags'].apply(lambda x: label_map[x[0]]).tolist())

#val_data['concatenated_text'] = val_data.apply(lambda row: ' '.join(row['postText']) + ' ' + ' '.join(row['targetTitle']) + ' ' + ' '.join(row['targetParagraphs']), axis=1)
postText_label = "postText"
targetTitle_label = "targetTitle"
targetParagraphs_label = "targetParagraphs"
val_data['concatenated_text'] = val_data.apply(
    lambda row: f"{postText_label} {' '.join(row['postText'])} "
                f"{targetTitle_label} {' '.join(row['targetTitle'])} "
                f"{targetParagraphs_label} {' '.join(row['targetParagraphs'])}",
    axis=1
)
val_inputs = val_data['concatenated_text'].tolist()
val_inputs = [preprocess_text(text) for text in val_inputs]
val_inputs = tokenizer(val_inputs, return_tensors='pt', padding=True, truncation=True, max_length=1024)  # Adjusted max_length

val_summaries = val_data['spoiler'].tolist()
val_labels = torch.tensor(val_data['tags'].apply(lambda x: label_map[x[0]]).tolist())

# Create DataLoaders for training and validation data
train_dataset = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
val_dataset = torch.utils.data.TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
# Evaluate on Training Set

# Lists to store predictions and labels
classification_predictions = []
summary_predictions = []
classification_labels = []
reference_summaries = train_summaries

# Evaluation loop

custom_model.eval()
with torch.no_grad():
    for batch in tqdm(train_loader, desc="Evaluating"):

        input_ids, attention_mask, labels = batch  # Assuming labels are in the batch

        # Ensure inputs are on the correct device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Text Summarization
        try:
            summary_ids = custom_model.transformer.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=50,
                num_beams=4,
                early_stopping=True
            )
            summary_texts = [
                tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True) for ids in summary_ids
            ]
            summary_predictions.extend(summary_texts)

            # Add the corresponding reference summaries
            reference_summaries.extend([tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True) for ids in labels])
        except Exception as e:
            print(f"Error generating summary: {e}")
            summary_texts = [""] * input_ids.size(0)
            summary_ids = torch.zeros((input_ids.size(0), 1), dtype=torch.long).to(device)
            summary_predictions.extend(summary_texts)

        # Classification
        outputs = custom_model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=summary_ids)
        logits = outputs['logits']
        predicted_label_idxs = torch.argmax(logits, dim=-1).tolist()

        label_map = {0: 'phrase', 1: 'passage', 2: 'multi'}
        classification_predictions.extend([label_map[idx] for idx in predicted_label_idxs])
        classification_labels.extend([label_map[idx] for idx in labels.tolist()])  # Assuming labels are in index format

# Compute Classification Metrics
f1 = f1_score(classification_labels, classification_predictions, average='weighted')
accuracy = accuracy_score(classification_labels, classification_predictions)

print(f"Classification F1 Score: {f1:.4f}")
print(f"Classification Accuracy: {accuracy:.4f}")

# Compute METEOR Scores
reference_summaries = [' '.join(tokens) if isinstance(tokens, list) else tokens for tokens in reference_summaries]
summary_predictions = [str(text) for text in summary_predictions]

meteor_scores = [
    meteor_score([word_tokenize(ref)], word_tokenize(gen))
    for ref, gen in zip(reference_summaries, summary_predictions)
]
average_meteor = sum(meteor_scores) / len(meteor_scores) if meteor_scores else 0

print(f"Average METEOR Score: {average_meteor:.4f}")



In [ ]:
# Evaluate on Validation Set

# Lists to store predictions and labels
classification_predictions = []
summary_predictions = []
classification_labels = []
reference_summaries = val_summaries

# Evaluation loop

custom_model.eval()
with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):

        input_ids, attention_mask, labels = batch  # Assuming labels are in the batch

        # Ensure inputs are on the correct device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Text Summarization
        try:
            summary_ids = custom_model.transformer.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=50,
                num_beams=4,
                early_stopping=True
            )
            summary_texts = [
                tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True) for ids in summary_ids
            ]
            summary_predictions.extend(summary_texts)

            # Add the corresponding reference summaries
            reference_summaries.extend([tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True) for ids in labels])
        except Exception as e:
            print(f"Error generating summary: {e}")
            summary_texts = [""] * input_ids.size(0)
            summary_ids = torch.zeros((input_ids.size(0), 1), dtype=torch.long).to(device)
            summary_predictions.extend(summary_texts)

        # Classification
        outputs = custom_model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=summary_ids)
        logits = outputs['logits']
        predicted_label_idxs = torch.argmax(logits, dim=-1).tolist()

        label_map = {0: 'phrase', 1: 'passage', 2: 'multi'}
        classification_predictions.extend([label_map[idx] for idx in predicted_label_idxs])
        classification_labels.extend([label_map[idx] for idx in labels.tolist()])  # Assuming labels are in index format

# Compute Classification Metrics
f1 = f1_score(classification_labels, classification_predictions, average='weighted')
accuracy = accuracy_score(classification_labels, classification_predictions)

print(f"Classification F1 Score: {f1:.4f}")
print(f"Classification Accuracy: {accuracy:.4f}")

# Compute METEOR Scores

reference_summaries = [' '.join(tokens) if isinstance(tokens, list) else tokens for tokens in reference_summaries]
summary_predictions = [str(text) for text in summary_predictions]

meteor_scores = [
    meteor_score([word_tokenize(ref)], word_tokenize(gen))
    for ref, gen in zip(reference_summaries, summary_predictions)
]
average_meteor = sum(meteor_scores) / len(meteor_scores) if meteor_scores else 0

print(f"Average METEOR Score: {average_meteor:.4f}")

